<a href="https://colab.research.google.com/github/vkt1414/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time
import ast

In [3]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Wed Apr 05 18:16:30 2023

Current directory :/content
Python version    : 3.9.16 (main, Dec  7 2022, 01:11:51) 


In [4]:
try:
  os.remove(f'{curr_dir}/TotalSegmentator.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py



--2023-04-05 18:16:32--  https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66062 (65K) [text/plain]
Saving to: ‘TotalSegmentator.py’

TotalSegmentator.py 100%[===================>]  64.51K  --.-KB/s    in 0.01s   

2023-04-05 18:16:32 (5.38 MB/s) - ‘TotalSegmentator.py’ saved [66062/66062]



In [11]:
with open('TotalSegmentator.py', 'r') as f:
    content = f.read()
    tree = ast.parse(content)

class TotalSegmentatorLogicFinder(ast.NodeVisitor):
    def __init__(self):
        self.result = None

    def visit_ClassDef(self, node):
        if node.name == 'TotalSegmentatorLogic':
            for body_node in node.body:
                if isinstance(body_node, ast.FunctionDef) and body_node.name == '__init__':
                    for expr in body_node.body:
                        if isinstance(expr, ast.Assign):
                            target = expr.targets[0]
                            if isinstance(target, ast.Attribute) and target.attr == 'totalSegmentatorLabelTerminology':
                                self.result = ast.literal_eval(ast.unparse(expr.value))
        self.generic_visit(node)

finder = TotalSegmentatorLogicFinder()
finder.visit(tree)
totalSegmentatorLabelTerminology = finder.result
data = []
for key, value in totalSegmentatorLabelTerminology.items():
    fields = value.split('~')
    row = {'Structure': key}
    for i, field in enumerate(fields):
        subfields = field.split('^')
        if i == 0:
            if len(subfields) > 1:
                row['SegmentAlgorithmName'] = subfields[0]
                row['SegmentAlgorithmType'] = subfields[1]
        elif i == 1:
            if len(subfields) > 2:
                row['SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyCategoryCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyCategoryCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 2:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 3:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'] = subfields[2]
    data.append(row)

df = pd.DataFrame(data)
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,spleen,SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,
1,kidney_right,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right
2,kidney_left,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left
3,gallbladder,SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,
4,liver,SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,
...,...,...,...,...,...,...,...,...,...,...
99,autochthon_left,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left
100,autochthon_right,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right
101,iliopsoas_left,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left
102,iliopsoas_right,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right


In [23]:
df1=df[['Structure',
       'SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator',
       'SegmentedPropertyCategoryCodeSequence.CodeValue',
       'SegmentedPropertyCategoryCodeSequence.CodeMeaning']]

df2=df[['Structure',
       'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
       'SegmentedPropertyTypeCodeSequence.CodeValue',
       'SegmentedPropertyTypeCodeSequence.CodeMeaning',
]]
df3=df[['Structure',
       
       'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
       'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
       'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'
]]    

In [26]:
import requests
import pandas as pd

base_url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/MAIN'
headers = { 'User-Agent': 'Python'}

# Replace this with the actual language reference set id for your desired language
language_refset = '900000000000509007'

# Create new columns for preferred and acceptable terms
# df1['PreferredTerms'] = None
# df1['AcceptableTerms'] = None

# Get unique SCT codes in column
unique_sct_codes = df2['SegmentedPropertyTypeCodeSequence.CodeValue'].unique()

# Get preferred and acceptable terms for each unique SCT code
sct_code_terms = {}
for sct_code in unique_sct_codes:
    # Get all descriptions for the given SCT code
    response = requests.get(f'{base_url}/descriptions?concept={sct_code}&limit=50', headers=headers)
    descriptions = response.json()['items']
    
    # Filter descriptions by acceptability and activity
    preferred_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'PREFERRED']
    acceptable_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'ACCEPTABLE']
    
    # Store terms for SCT code
    sct_code_terms[sct_code] = (preferred_terms, acceptable_terms)

# # Add arrays to dataframe
# for index, row in df1.iterrows():
#     sct_code = row['SegmentedPropertyCategoryCodeSequence.CodeValue']
#     preferred_terms, acceptable_terms = sct_code_terms[sct_code]
#     df1.at[index, 'PreferredTerms'] = preferred_terms
#     df1.at[index, 'AcceptableTerms'] = acceptable_terms

sct_code_terms

{'78961009': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '64033007': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '28231008': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '10200004': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '69695003': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '15825003': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '64131007': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '32764006': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '15776009': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '23451007': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '45653009': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],
  []),
 '90572001': (['Bronchocele', 'Ulceroglandular tularemia', 'Auscultoscope'],

In [13]:
import requests
import pandas as pd

base_url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/MAIN'

# Replace these with the actual column names from your dataframe
sct_codes = ['SegmentedPropertyTypeCodeSequence.CodeValue', 'SegmentedPropertyTypeModifierCodeSequence.CodeValue', 'SegmentedPropertyCategoryCodeSequence.CodeValue']

# Replace this with the actual language reference set id for your desired language
language_refset = '900000000000509007'
headers = { 'User-Agent': 'Python'}

for sct_code_column in sct_codes:
    # Create new columns for preferred and acceptable terms
    preferred_terms_column = sct_code_column + '.PreferredTerms'
    acceptable_terms_column = sct_code_column + '.AcceptableTerms'
    df[preferred_terms_column] = None
    df[acceptable_terms_column] = None
    
    # Get unique SCT codes in column
    unique_sct_codes = df[sct_code_column].unique()
    
    # Get preferred and acceptable terms for each unique SCT code
    sct_code_terms = {}
    for sct_code in unique_sct_codes:
        # Get all descriptions for the given SCT code
        response = requests.get(f'{base_url}/descriptions?concept={sct_code}&limit=50', headers=headers)
        descriptions = response.json()['items']
        
        # Filter descriptions by acceptability and activity
        preferred_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'PREFERRED']
        acceptable_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'ACCEPTABLE']
        
        # Store terms for SCT code
        sct_code_terms[sct_code] = (preferred_terms, acceptable_terms)
    
    # Add arrays to dataframe
    for index, row in df.iterrows():
        sct_code = row[sct_code_column]
        preferred_terms, acceptable_terms = sct_code_terms[sct_code]
        df.at[index, preferred_terms_column] = preferred_terms
        df.at[index, acceptable_terms_column] = acceptable_terms

df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms,SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms,SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms,SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms
0,spleen,SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
1,kidney_right,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
2,kidney_left,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
3,gallbladder,SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
4,liver,SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,autochthon_left,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
100,autochthon_right,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
101,iliopsoas_left,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]
102,iliopsoas_right,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right,"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[],"[Bronchocele, Ulceroglandular tularemia, Auscu...",[]


In [43]:
import requests
import pandas as pd
url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/MAIN/2019-07-31/descriptions?conceptId=78961009&acceptabilityMap.900000000000509007=PREFERRED'
headers = { 'User-Agent': 'Python'}

response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    data = response.json()
    # Load the data into a DataFrame
    df = pd.json_normalize(data['items'])
else:
    print(f'An error occurred: {response.status_code}')
df

,active,moduleId,released,releasedEffectiveTime,descriptionId,term,conceptId,typeId,type,lang,caseSignificance,effectiveTime,acceptabilityMap.900000000000509007,acceptabilityMap.900000000000508004
0,True,900000000000207008,True,20170731,131023013,Spleen,78961009,900000000000013009,SYNONYM,en,CASE_INSENSITIVE,20170731,ACCEPTABLE,ACCEPTABLE
1,True,900000000000207008,True,20170731,503973011,Splenic structure,78961009,900000000000013009,SYNONYM,en,CASE_INSENSITIVE,20170731,PREFERRED,PREFERRED
2,False,900000000000207008,True,20020131,131024019,"Spleen, NOS",78961009,900000000000013009,SYNONYM,en,INITIAL_CHARACTER_CASE_INSENSITIVE,20020131,NaN,NaN
3,True,900000000000207008,True,20170731,819959017,Splenic structure (body structure),78961009,900000000000003001,FSN,en,CASE_INSENSITIVE,20170731,PREFERRED,PREFERRED


In [45]:
from urllib.request import urlopen, Request
from urllib.parse import quote
import json

baseUrl = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct'
edition = 'MAIN'
version = '2019-07-31'

def urlopen_with_header(url):
    # adds User-Agent header otherwise urlopen on its own gets an IP blocked response
    req = Request(url)
    req.add_header('User-Agent','Python')
    return urlopen(req)

#Prints fsn of a concept
def getConceptById(id):
    url = baseUrl + '/browser/' + edition + '/' + version + '/concepts/' + id
    response = urlopen_with_header(url).read()
    data = json.loads(response.decode('utf-8'))
    print(url)
    print (data['fsn']['term'])

#Prints description by id
def getDescriptionById(id):
    url = baseUrl + '/' + edition + '/' + version + '/descriptions/' + id
    response = urlopen_with_header(url).read()
    data = json.loads(response.decode('utf-8'))

    print (data['term'])

#Prints number of concepts with descriptions containing the search term
def getConceptsByString(searchTerm):
    url = baseUrl + '/browser/' + edition + '/' + version + '/concepts?term=' + quote(searchTerm) + '&activeFilter=true&offset=0&limit=50'
    response = urlopen_with_header(url).read()
    data = json.loads(response.decode('utf-8'))

    print (data['total'])

#Prints number of descriptions containing the search term with a specific semantic tag
def getDescriptionsByStringFromProcedure(searchTerm, semanticTag):
    url = baseUrl + '/browser/' + edition + '/' + version + '/descriptions?term=' + quote(searchTerm) + '&conceptActive=true&semanticTag=' + quote(semanticTag) + '&groupByConcept=false&searchMode=STANDARD&offset=0&limit=50'
    response = urlopen_with_header(url).read()
    data = json.loads(response.decode('utf-8'))

    print (data['totalElements'])

#getConceptById('78961009')
#getDescriptionById('78961009')
# getConceptsByString('heart attack')
# getDescriptionsByStringFromProcedure('heart', 'procedure')